## memo_re 0 init

### to version = "memo_0"
```
초기화 과정입니다.
```

In [27]:
#01,13,21,25,28,31
candidate = {0:1, 1:13, 2:21, 3:25, 4:28, 5:31}
next_round = 1186

In [28]:
## this code was run.
#!pip install keras-tuner
# import numpy as np
# import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
# print(f'Current Step #01 [now = {datetime.now()}')
from IPython.display import Markdown

from lib.analy_util import ( draw_markdown,
                             draw_graph,
                             draw_scatter,
                             get_pre_bef,
                             draw_average,
                             get_frequency,
                             get_information,
                             analyze_v1,
                             data_prepare,
                             search_model,
                             get_information_v2,
                             get_extdatas,
                             )
print(f'define this code. [{datetime.now()}]')


to_version="memo_0818_5"
next_round = 1186

title = "Metrics"
row_length = 400
new_to_old = True
show_mark_down = True
markdown_table_limit_length = 5

define this code. [2025-08-18 18:48:52.734354]


In [1]:
create_query = f'create table if not exists results_order(id INTEGER PRIMARY KEY AUTOINCREMENT, round short, n1 short, n2 short, n3 short, n4 short, n5 short, n6 short, bonus short);'
!echo "{create_query}" | sqlite3 ../db/metrics.db

In [33]:
"""
init memo_re
"""

datas = """
1185,17,6,32,28,29,22,38
1184,16,14,23,31,37,25,42
1183,36,4,15,23,17,27,31
1182,31,21,13,25,28,1,22
1181,8,14,20,41,10,33,28
1180,37,40,18,6,12,41,3
1179,44,18,16,24,40,3,21
1178,43,5,27,44,6,11,17
1177,3,43,7,15,19,16,21
1176,30,11,35,9,7,21,29
1175,42,8,4,3,6,32,31
1174,8,17,39,14,36,11,22
1173,30,35,20,1,18,5,3
1172,44,42,40,9,24,7,45
1171,7,3,12,6,17,11,19
1170,3,38,13,28,34,42,25
1169,39,26,24,12,42,5,20
1168,33,9,24,37,21,30,29
1167,35,40,8,31,39,23,24
1166,14,42,25,29,23,27,16
1165,38,29,45,7,6,27,17
1164,25,18,39,23,17,38,22
1163,13,2,33,16,15,43,4
1162,29,28,21,25,22,20,6
1161,2,12,34,24,20,42,37
1160,39,7,36,13,45,18,19
1159,3,28,39,9,38,27,7
1158,38,21,27,25,32,37,20
1157,5,7,20,26,25,12,28
1156,45,39,34,41,30,31,7
1155,19,10,37,27,38,16,13
1154,22,38,32,8,4,26,27
1153,35,9,44,13,1,10,5
1152,31,36,35,37,32,30,5
1151,27,29,9,3,15,2,8
1150,45,18,39,35,9,8,25
1149,21,15,8,32,36,19,38
1148,16,22,13,15,6,3,32
1147,7,37,11,26,24,27,32
1146,6,19,17,11,40,43,28
1145,31,33,2,11,44,37,32
1144,34,12,4,26,3,15,6
1143,36,17,27,16,10,28,6
1142,37,41,30,2,28,8,22
1141,21,26,35,12,7,11,20
1140,38,29,10,22,31,7,15
"""

histories = datas.split("\n")
for history in histories:
    cols = history.split(',')
    if len(cols) == 8:
        sel_query = f'select * from results_order where round={cols[0]} and n1={cols[1]} and n1={cols[1]} and n2={cols[2]} and n3={cols[3]} and n4={cols[4]} and n5={cols[5]} and n6={cols[6]};'
        results = !echo "{sel_query}" | sqlite3 ../db/metrics.db
        if len(results) == 0:
            insert_query = f'insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values({cols[0]}, {cols[1]}, {cols[2]}, {cols[3]}, {cols[4]}, {cols[5]}, {cols[6]}, {cols[7]});'
            !echo "{insert_query}" | sqlite3 ../db/metrics.db
